In [2]:
import requests
import json
import time
from datetime import date
import Utils as utils
import pandas as pd
import _thread

## Run measurements

In [3]:
df = utils.getAvailablePCProbesCC()

df = df.loc[df['CountryCode'].isin(utils.ccs)]
    
available_ccs = (df.groupby(['CountryCode']).size().to_frame(name = 'count').reset_index()).CountryCode.values

df.loc[df['CountryCode']=='ZA'].head()


,ASN,CityName,ConnectionType,CountryCode,DNSResolver,GeolocationAccuracy,IPAddress,Latitude,Longitude,Network,NetworkID,Platform,ProbeID,Screensize,Version
3,29975,Pretoria,Ethernet,ZA,41.1.161.253,10000,41.13.68.19,-25.76,28.14,Vodacom,1045,PC,a19b37cd-c270-4299-b28f-4e251b25ea8a,1280x768,1.0.44.0
11,37457,Pretoria,Wireless80211,ZA,196.43.45.231,1000,102.250.0.72,-26.71,27.13,Telkom Internet,422,PC,5e35f97d-23e0-48b0-b1ef-97e337443eba,1920x1080,1.0.44.0
22,37457,Johannesburg,Ethernet,ZA,196.43.42.230,10000,105.225.149.20,-26.23,28.06,Telkom Internet,422,PC,627b3e16-ba7d-4768-a121-28017acbf3a8,1920x1080,1.0.45.0
24,37457,Pretoria,Ethernet,ZA,196.43.50.229,5000,105.184.27.126,-25.76,28.14,Telkom Internet,422,PC,cea2de8f-4491-4e6a-84e9-578618b6a303,1280x1024,1.0.44.0
35,37501,Unknown,"Wireless80211,Tunnel",ZA,146.182.9.12,25000,196.255.202.173,-29.12,26.17,UOFS,30463,PC,be585e8c-b08f-43f1-904e-248dbb8977e1,1366x768,1.0.44.0


In [3]:
df.loc[df['CountryCode']=='ZA'].head()

,ASN,CityName,ConnectionType,CountryCode,DNSResolver,GeolocationAccuracy,IPAddress,Latitude,Longitude,Network,NetworkID,Platform,ProbeID,Screensize,Version
34,10474,Kempton Park,"Ethernet,Tunnel",ZA,196.28.182.24,10000,197.88.46.48,-26.18,28.32,MWEB,1300,PC,4b23051b-6b30-4c87-86a9-499288cde1de,1366x768,1.0.44.0
36,29975,Cape Town,Wireless80211,ZA,41.1.47.253,10000,41.13.250.61,-33.92,18.42,Vodacom,1045,PC,02913d85-84d0-4e0b-a309-ac04432a8d19,NaN,1.0.46.1
53,36937,Pretoria,Ethernet,ZA,165.73.227.253,25000,165.73.225.12,-24.60,27.40,WAN 4 U cc,105679,PC,b7f37d52-c167-4469-becb-725be1034fa5,1366x768,1.0.44.0
55,36937,Johannesburg,Ethernet,ZA,61.220.8.173,25000,41.174.138.57,-26.23,28.06,Neotel Pty Ltd,6925,PC,37e7858a-0ff8-4fdc-8d15-14c6e4ef92ce,NaN,1.0.46.1
59,37457,Pietermaritzburg,Wireless80211,ZA,196.43.53.231,1000,105.227.210.68,-29.61,30.39,Telkom Internet,422,PC,a3f72fa7-1a93-4eab-8d3b-6e664f358c1b,NaN,1.0.44.0


In [4]:
utils.getProbesByCC(df,'ZA')

[{'ProbeID': '4b23051b-6b30-4c87-86a9-499288cde1de',
  'Latitude': -26.18,
  'Longitude': 28.32,
  'Screensize': '1366x768'},
 {'ProbeID': '02913d85-84d0-4e0b-a309-ac04432a8d19',
  'Latitude': -33.92,
  'Longitude': 18.42,
  'Screensize': nan},
 {'ProbeID': 'b7f37d52-c167-4469-becb-725be1034fa5',
  'Latitude': -24.6,
  'Longitude': 27.4,
  'Screensize': '1366x768'},
 {'ProbeID': '37e7858a-0ff8-4fdc-8d15-14c6e4ef92ce',
  'Latitude': -26.23,
  'Longitude': 28.06,
  'Screensize': nan},
 {'ProbeID': 'a3f72fa7-1a93-4eab-8d3b-6e664f358c1b',
  'Latitude': -29.61,
  'Longitude': 30.39,
  'Screensize': nan},
 {'ProbeID': 'caa48bca-201b-40b7-a177-cc16779e6bc4',
  'Latitude': -33.03,
  'Longitude': 27.92,
  'Screensize': '1024x768'},
 {'ProbeID': '0d87fdae-b9c9-4b02-9800-6c5b8c0ae855',
  'Latitude': -28.73,
  'Longitude': 24.77,
  'Screensize': '1920x1080'},
 {'ProbeID': '5ea5b97c-5602-4c0b-b0cc-ee79e68bdff0',
  'Latitude': -26.23,
  'Longitude': 28.06,
  'Screensize': '1920x1080'}]

In [5]:
for cc in available_ccs:

    probes = df.loc[df['CountryCode']==cc]

    if (probes is None):
        continue
    
    df_testID = pd.DataFrame(columns=['videoID', 'cc', 'testID', 'probeID', 'Latitude', 'Longitude', 'Resolution'])
    
    for index, p in probes.iterrows():
        videos = utils.getVideoList(str(p['Latitude']), str(p['Longitude']), '10km', 5)
        #print(videos)

        for v in videos:

            for res in ['SMALL_RES', 'MEDIUM_RES', 'LARGE_RES', 'HD720_RES', 'HD1080_RES']:

                testID = utils.runVideoTest(v, p['ProbeID'], res)

                df_testID = df_testID.append({'videoID': v, 'cc': cc, 'testID': testID, 
                                              'probeID': p['ProbeID'], 'Latitude': p['Latitude'], 
                                              'Longitude' : p['Longitude'], 'Resolution' : res}, ignore_index=True)

                print(v, cc, testID, p['ProbeID'], p['Latitude'], p['Longitude'], res)
        
df_testID.to_csv('data/testID-24052019.csv', sep='|', encoding='utf-8', index=False)   



Tg0yfKQQVF4 BI c0df40c1-7f8a-41ad-8a6c-38ca67c868f3 e078462a-e8dc-4d49-9d4f-14d72fee451b -3.39 29.37 SMALL_RES
Tg0yfKQQVF4 BI 49468ed4-0f4d-47a3-8628-09582f69b88f e078462a-e8dc-4d49-9d4f-14d72fee451b -3.39 29.37 MEDIUM_RES
Tg0yfKQQVF4 BI e8f7caa6-158e-4d02-865f-b5f50ac422a6 e078462a-e8dc-4d49-9d4f-14d72fee451b -3.39 29.37 LARGE_RES
Tg0yfKQQVF4 BI acd9112c-0c0d-4868-ad71-39c3dbe1a928 e078462a-e8dc-4d49-9d4f-14d72fee451b -3.39 29.37 HD720_RES
Tg0yfKQQVF4 BI ad552475-edaf-46bf-8035-35b736688bb5 e078462a-e8dc-4d49-9d4f-14d72fee451b -3.39 29.37 HD1080_RES
a0SDJ7xao4I BI 2aaa10a3-7f35-4f35-b83e-7091e166cff5 e078462a-e8dc-4d49-9d4f-14d72fee451b -3.39 29.37 SMALL_RES
a0SDJ7xao4I BI 72c6f651-1328-4a41-a2ba-0b3de5baab2e e078462a-e8dc-4d49-9d4f-14d72fee451b -3.39 29.37 MEDIUM_RES
a0SDJ7xao4I BI 411712a0-c36b-4c2e-b86b-ca4e0e284eaa e078462a-e8dc-4d49-9d4f-14d72fee451b -3.39 29.37 LARGE_RES
a0SDJ7xao4I BI 726efc81-44b4-4d5e-9580-1218cf3e4e6b e078462a-e8dc-4d49-9d4f-14d72fee451b -3.39 29.37 HD720_RE

## Collect results

In [2]:
print(utils.getAvailablePCProbesCC())

['BI' 'BW' 'DJ' 'DZ' 'EG' 'ET' 'KE' 'KM' 'LY' 'MA' 'NA' 'NG' 'SN' 'SO'
 'TN' 'ZA' 'ZW']


In [2]:
print(utils.getProbes('MU'))

[{'ProbeID': 'fb94eede-9aff-43fa-96ee-1f944129fcda', 'Latitude': -20.28, 'Longitude': 57.55}, {'ProbeID': '14a8ea0a-ff4a-4538-9880-e6fbfb12d8da', 'Latitude': -20.11, 'Longitude': 57.58}, {'ProbeID': 'af99e32f-37d4-466c-a807-bb93f91f565c', 'Latitude': -20.24, 'Longitude': 57.48}]


In [2]:
print(utils.runVideoTest('dkBoUC0RAT0', '14a8ea0a-ff4a-4538-9880-e6fbfb12d8da'))

589ddfac-2322-438c-83f9-8405f25713e7


In [2]:
print(utils.getProbes('MU'))

None


In [ ]:
retrieveVideoTestResults('5ead3b75-e21f-4390-a4bd-0e9799aa05d0')

In [5]:
print(utils.runVideoTest('8aYpKlhXDvE', 'b7f37d52-c167-4469-becb-725be1034fa5', 'NOT_SPECIFIED_RES'))

http://pcsucdn.com/YoutubeVideoTest2.html?v=8aYpKlhXDvE

